- optimize model by overall metrics * 
- utilize indoorunifiedwifids * 
- fill train-only and test-only labels as "only" 

In [1]:
# credit: https://www.kaggle.com/kokitanisaka/lstm-by-keras-with-unified-wi-fi-feats
!pip install tensorflow-determinism

  Created wheel for tensorflow-determinism: filename=tensorflow_determinism-0.1.0-py3-none-any.whl size=5255 sha256=f4e970eb5af860d4b123e06248148a2316bb474cbbb56577567ea6c588cca7b4
  Stored in directory: /root/.cache/pip/wheels/ce/06/d4/df757adc4c81f705a74a0317c27cf19919ccd25ae2a6ecd2c5
Successfully built tensorflow-determinism


In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from pathlib import Path
import glob
import pickle

import random
import os

from tqdm.notebook import tqdm

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler, QuantileTransformer

import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import tensorflow.keras.backend as K
import tensorflow_addons as tfa
from tensorflow_addons.layers import WeightNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.metrics import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *

import multiprocessing
import scipy.interpolate
import scipy.sparse
from tqdm import tqdm
from tensorflow_addons.layers import WeightNormalization

from sklearn.decomposition import PCA

from scipy.spatial.distance import cdist

import warnings
warnings.filterwarnings("ignore")

In [3]:
# options
N_SPLITS = 5
SEED = 2021
NUM_FEATS = 30 
base_path = '/kaggle'

In [4]:
def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    np.random.seed(seed)
    tf.random.set_seed(seed)
    session_conf = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1
    )
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)
    
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]

def xy_comp_metric(xhat, yhat, x, y):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2))
    return intermediate.sum()/xhat.shape[0]

def xy_loss(true, pred):
    return K.mean(K.sqrt(K.square(pred[:,0] - true[:,0]) + K.square(pred[:,1] - true[:,1]))) 

In [5]:
# indooruniteddataset: my original wifi data with additional 24 sited ?
# indoor-mod-united-dataset: my another wifi data with all sites x
# indoorunifiedwifids: kouki's wifi data ?
# indoor-mod-interpolated-united-data: my 3rd wifi data with 20 additional sites with interpolation x
# indoor-wifi-from-waypoint-time-dataset: my 4th wifi data from waypoint time with all sites ?

feature_dir = f"{base_path}/input/indoorunifiedwifids"   
train_files = sorted(glob.glob(os.path.join(feature_dir, '*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_dir, '*_test.csv')))
subm = pd.read_csv(f'{base_path}/input/indoor-location-navigation/sample_submission.csv', index_col=0)

all_sites = sorted(set(pd.DataFrame(subm.index)["site_path_timestamp"].apply(lambda x: x.split("_")[0])))

In [6]:
# training target features
BSSID_FEATS = [f'bssid_{i}' for i in range(NUM_FEATS)]
RSSI_FEATS  = [f'rssi_{i}' for i in range(NUM_FEATS)]

In [7]:
data = []
for i in tqdm(train_files):
    tmp = pd.read_csv(i)
    tmp["site_id"] = i.split("/")[4].split("_")[0]
    data.append(tmp)
data = pd.concat(data).reset_index(drop=True)

test_data = []
for i in tqdm(test_files):
    tmp = pd.read_csv(i)
    tmp["site_id"] = i.split("/")[4].split("_")[0]
    test_data.append(tmp)
test_data = pd.concat(test_data).reset_index(drop=True)

100%|██████████| 24/24 [00:03<00:00,  7.61it/s]


In [8]:
train_only_index = data[~data.site_id.isin(all_sites)].index
normal_index = data[data.site_id.isin(all_sites)].index
assert len(set(train_only_index) & set(normal_index)) == 0

In [9]:
wifi_feat_nums = 60
wifi_bssids = []
for i in range(wifi_feat_nums):
    wifi_bssids.extend(data.iloc[:,i].values.tolist())
wifi_bssids = list(set(wifi_bssids))

#wifi_bssids_size = len(wifi_bssids)
#print(f'BSSID TYPES: {wifi_bssids_size}')

wifi_bssids_test = []
for i in range(wifi_feat_nums):
    wifi_bssids_test.extend(test_data.iloc[:,i].values.tolist())
wifi_bssids_test = list(set(wifi_bssids_test))

#wifi_bssids_size = len(wifi_bssids_test)
#print(f'BSSID TYPES: {wifi_bssids_size}')

train_only_bssid = list(set(wifi_bssids) - set(wifi_bssids_test))
test_only_bssid = list(set(wifi_bssids_test) - set(wifi_bssids))
both_bssid = list(set(wifi_bssids) & set(wifi_bssids_test))
print(len(train_only_bssid), len(test_only_bssid), len(both_bssid))

#wifi_bssids.extend(wifi_bssids_test)
#wifi_bssids_size = len(wifi_bssids)

both_bssid.extend(["only"])
wifi_bssids = both_bssid.copy()
wifi_bssids_size = len(wifi_bssids)
print(wifi_bssids_size)

27868 226 28394
28395


In [10]:
for i in range(NUM_FEATS):
    #print(np.sum(data["bssid_"+str(i)].isin(train_only_bssid)))
    data.loc[data["bssid_"+str(i)].isin(train_only_bssid), "bssid_"+str(i)] = "only"
    #print(np.sum(data["bssid_"+str(i)].isin(train_only_bssid)))
    
for i in range(NUM_FEATS):
    #print(np.sum(test_data["bssid_"+str(i)].isin(test_only_bssid)))
    test_data.loc[test_data["bssid_"+str(i)].isin(test_only_bssid), "bssid_"+str(i)] = "only"
    #print(np.sum(test_data["bssid_"+str(i)].isin(test_only_bssid)))

In [11]:
#floor_pred = pd.read_csv('../input/lstm-prediction-dataset/submission.csv') 
#test_data['floor'] = floor_pred['floor'].values

original_floor_values = data.loc[normal_index]["floor"].values

data = pd.get_dummies(data, columns=['floor'])
#test_data = pd.get_dummies(test_data, columns=['floor'])

floor_columns = [i for i in data.columns if "floor" in i]

In [12]:
train_only_site_id = list(set(data['site_id'].unique()) - set(test_data['site_id'].unique()))
print(len(train_only_site_id))
data.loc[data["site_id"].isin(train_only_site_id), "site_id"] = "only"

0


In [13]:
# preprocess
ss = StandardScaler()
ss.fit(data.loc[:,RSSI_FEATS])

le = LabelEncoder()
le.fit(wifi_bssids)

le_site = LabelEncoder()
le_site.fit(data['site_id'])

LabelEncoder()

In [14]:
data.loc[:,RSSI_FEATS] = ss.transform(data.loc[:,RSSI_FEATS])
test_data.loc[:,RSSI_FEATS] = ss.transform(test_data.loc[:,RSSI_FEATS])

for i in BSSID_FEATS:
    data.loc[:,i] = le.transform(data.loc[:,i])
    
for i in BSSID_FEATS:
    test_data.loc[:,i] = le.transform(test_data.loc[:,i])
    
data.loc[:, 'site_id'] = le_site.transform(data.loc[:, 'site_id'])
test_data.loc[:, 'site_id'] = le_site.transform(test_data.loc[:, 'site_id'])

In [15]:
site_count = len(data['site_id'].unique())
data.reset_index(drop=True, inplace=True)

In [16]:
set_seed(SEED)

In [17]:
train_only_data = data[data.index.isin(train_only_index)].copy().reset_index(drop=True)
data = data[data.index.isin(normal_index)].copy().reset_index(drop=True)
data.shape, train_only_data.shape

((258125, 215), (0, 215))

# model

In [18]:
print(tf.test.gpu_device_name())

/device:GPU:0


In [19]:
bssid_embed_dim = 64
site_embed_dim = 2
batch_size=256
epochs=50

In [20]:
def conv_layer(x, n_channels=32, kernel_size=3, pool_size=2, padding='same'):
    x = Conv1D(filters=n_channels, kernel_size=kernel_size, padding=padding)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPool1D(pool_size=pool_size, padding='same')(x)
    return x

def create_model(input_data):

    # bssid feats ------
    input_dim = input_data[0].shape[1]

    input_embd_layer = L.Input(shape=(input_dim,))
    x1 = L.Embedding(wifi_bssids_size, bssid_embed_dim)(input_embd_layer)
    x1 = L.Flatten()(x1)

    # rssi feats -------
    input_dim = input_data[1].shape[1]
    
    ## cnn
    input_layer = Input(shape=(input_dim, 1, 1))
    x2 = Reshape((input_dim, 1))(input_layer)
    x2 = conv_layer(x2, 2, 7, 3)  
    x2 = conv_layer(x2, 2, 7, 3) 
    x2 = Reshape((-1,))(x2) #x2 = GlobalMaxPool1D()(x2)
    
    # site -------------
    input_site_layer = L.Input(shape=(1,))
    x3 = L.Embedding(site_count, site_embed_dim)(input_site_layer)
    x3 = L.Flatten()(x3)

    # main stream ------
    x = L.Concatenate(axis=1)([x1, x3, x2])
    
    x = L.BatchNormalization()(x)
    x = L.Dropout(0.3)(x)
    x = L.Dense(256, activation='relu')(x)
    
    x = L.Reshape((1, -1))(x)
    x = L.BatchNormalization()(x)
    x = L.Bidirectional(L.LSTM(128, dropout=0.3, recurrent_dropout=0.3, return_sequences=True, activation='elu'))(x)
    x = L.Bidirectional(L.LSTM(16, return_sequences=False, activation='elu'))(x)
        
    output_layer_1 = L.Dense(2, name='xy')(x)
    output_layer_2 = L.Dense(len(floor_columns), activation='softmax', name='floor')(x)

    model = M.Model([input_embd_layer, input_layer, input_site_layer], 
                    [output_layer_1, output_layer_2])
    
    model.compile(optimizer=tf.optimizers.Adam(lr=0.002),
                  loss={'xy': xy_loss, 'floor': tf.keras.losses.CategoricalCrossentropy()}, 
                  loss_weights={'xy': 1, 'floor': 15},
                  metrics={'xy': xy_loss, 'floor': 'accuracy'})

    return model

In [21]:
score_df = pd.DataFrame()
predictions = list()

oof_xy = list()
oof_x, oof_y, oof_f = np.zeros(data.shape[0]), np.zeros(data.shape[0]), np.zeros(data.shape[0])
preds_x, preds_y = 0, 0
preds_f_arr = np.zeros((test_data.shape[0], N_SPLITS))

for fold, (trn_idx, val_idx) in enumerate(StratifiedKFold(n_splits=N_SPLITS, shuffle=True, 
                                                          random_state=SEED).split(data.loc[:, 'path'], 
                                                                                   data.loc[:, 'path'])):
        
    X_train = pd.concat([data.loc[trn_idx, BSSID_FEATS + RSSI_FEATS + ['site_id'] ], 
                        train_only_data[BSSID_FEATS + RSSI_FEATS + ['site_id']]], axis=0).reset_index(drop=True) 
    y_trainx = pd.concat([data.loc[trn_idx, 'x'], train_only_data['x']], axis=0).reset_index(drop=True)
    y_trainy = pd.concat([data.loc[trn_idx, 'y'], train_only_data['y']], axis=0).reset_index(drop=True)
    y_trainf = pd.concat([data.loc[trn_idx][floor_columns], train_only_data[floor_columns]], axis=0).reset_index(drop=True)
    
    tmp = pd.concat([y_trainx, y_trainy], axis=1)
    y_train = [tmp, y_trainf]

    X_valid = data.loc[val_idx, BSSID_FEATS + RSSI_FEATS + ['site_id'] ] 
    y_validx = data.loc[val_idx, 'x']
    y_validy = data.loc[val_idx, 'y']
    y_validf = data.loc[val_idx][floor_columns]

    tmp = pd.concat([y_validx, y_validy], axis=1)
    y_valid = [tmp, y_validf]
    
    model = create_model([X_train.loc[:,BSSID_FEATS], X_train.loc[:,RSSI_FEATS], X_train.loc[:,'site_id']])
    model.fit([X_train.loc[:,BSSID_FEATS], X_train.loc[:,RSSI_FEATS], X_train.loc[:,'site_id']], y_train,   
                validation_data=([X_valid.loc[:,BSSID_FEATS], X_valid.loc[:,RSSI_FEATS], X_valid.loc[:,'site_id']], y_valid), 
                batch_size=batch_size, epochs=epochs,
                callbacks=[
                ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_delta=1e-4, mode='min')
                ,ModelCheckpoint(f'RNN_{SEED}_{fold}.hdf5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min')
                ,EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=5, mode='min', baseline=None, restore_best_weights=True)
            ])

    model.load_weights(f'RNN_{SEED}_{fold}.hdf5')  
    val_pred = model.predict([X_valid.loc[:,BSSID_FEATS], X_valid.loc[:,RSSI_FEATS], X_valid.loc[:,'site_id']]) 

    oof_x[val_idx] = val_pred[0][:,0]
    oof_y[val_idx] = val_pred[0][:,1]
    oof_f[val_idx] = np.argmax(val_pred[1], axis=1)-2
                                                            
    pred = model.predict([test_data.loc[:,BSSID_FEATS], test_data.loc[:,RSSI_FEATS], test_data.loc[:,'site_id']]) 
    preds_x += pred[0][:,0]
    preds_y += pred[0][:,1]
    preds_f_arr[:, fold] = np.argmax(pred[1], axis=1)-2

    score = comp_metric(oof_x[val_idx], oof_y[val_idx], oof_f[val_idx],
                        y_validx.to_numpy(), y_validy.to_numpy(), original_floor_values[val_idx])
    print(f"fold {fold}: overall metrics {score}")
    score2 = xy_comp_metric(oof_x[val_idx], oof_y[val_idx],
                        y_validx.to_numpy(), y_validy.to_numpy())
    print(f"fold {fold}: root mean xy position error {score2}")
    oof_xy.append(score2)

    K.clear_session()
    
preds_x /= (fold + 1)
preds_y /= (fold + 1)
    
print("*+"*40)
score = comp_metric(oof_x, oof_y, oof_f, data["x"].to_numpy(), data["y"].to_numpy(), original_floor_values)
print(f"overall metrics {score}")
score2 = xy_comp_metric(oof_x, oof_y, data["x"].to_numpy(), data["y"].to_numpy())
print(f"root mean xy position error {score2}")
print(oof_xy)
print("*+"*40)

preds_f_mode = stats.mode(preds_f_arr, axis=1)
preds_f = preds_f_mode[0].astype(int).reshape(-1)
test_preds = pd.DataFrame(np.stack((preds_f, preds_x, preds_y))).T
test_preds.columns = subm.columns
test_preds.index = test_data["site_path_timestamp"]
test_preds["floor"] = test_preds["floor"].astype(int)
predictions.append(test_preds)

Epoch 1/50
807/807 [==============================] - 70s 70ms/step - loss: 79.6716 - xy_loss: 68.4752 - floor_loss: 0.7464 - xy_xy_loss: 68.4752 - floor_accuracy: 0.7430 - val_loss: 19.3968 - val_xy_loss: 16.8858 - val_floor_loss: 0.1674 - val_xy_xy_loss: 16.8846 - val_floor_accuracy: 0.9405

Epoch 00001: val_loss improved from inf to 19.39682, saving model to RNN_2021_0.hdf5
Epoch 2/50
807/807 [==============================] - 56s 69ms/step - loss: 21.4540 - xy_loss: 18.7561 - floor_loss: 0.1799 - xy_xy_loss: 18.7561 - floor_accuracy: 0.9325 - val_loss: 17.3669 - val_xy_loss: 15.2445 - val_floor_loss: 0.1415 - val_xy_xy_loss: 15.2413 - val_floor_accuracy: 0.9428

Epoch 00002: val_loss improved from 19.39682 to 17.36687, saving model to RNN_2021_0.hdf5
Epoch 3/50
807/807 [==============================] - 54s 67ms/step - loss: 19.0376 - xy_loss: 16.6818 - floor_loss: 0.1571 - xy_xy_loss: 16.6818 - floor_accuracy: 0.9401 - val_loss: 17.0288 - val_xy_loss: 14.7648 - val_floor_loss: 0.1

KeyError: 'site_path_timestamp'

In [22]:
all_preds = pd.concat(predictions)
all_preds = all_preds.reindex(subm.index)
all_preds.to_csv('submission_bef_pp.csv')

ValueError: No objects to concatenate